In [1]:
import os
import re

In [11]:
def jpg_image(dirname):
    try:
    
        direct = os.listdir(dirname)
        jpg_list = []
        for file in direct:
            full_file = os.path.join(dirname, file)

            if os.path.isdir(full_file):
                jpg_image(full_file)

            else:
                ext = os.path.splitext(full_file)[-1]


                if ext == '.jpg':
                    jpg_list.append(full_file)
            print(jpg_list)

            return jpg_list
    
    except PermissionError:
        pass

In [12]:
jpg_image('D:Data_collection/mask_data/')

[]
[]


[]